# Projective Preferential Prior Elicitation: DSGE prior elicitation

<font size="4">In this notebook the expert knowledge about DSGE priors can be elicited by using the [Projective Preferential Bayesian Optimization](https://arxiv.org/abs/2002.03113) framework. The test case is a New Keynesian pre-crisis DSGE model from Smets and Wouters (2007) as it appears in (Fratto and Uhlig, 2019). </font> 

<font size="4"><font color="red">**INSTRUCTIONS**:
- Read [this](../files/dsge/docs/Instructions.pdf) first
- Write your name within the apostrophe:</font> </font> 

In [1]:
SESSION_NAME = 'PETRUS'

<font size="4"><font color="red">
- Click: 'Kernel' $\rightarrow$ 'Restart & Run All' $\rightarrow$ 'Restart and Run All Cells'
- Scroll down to the cell below 'Elicitation loop' and wait until you can provide feedback
- Wait and provide your feedback until 'Elicitation done!' is printed
- Your input is no longer needed, thank you! However, please do not logout until 'Thank you for your effort!...' is printed.</font> </font> 

#### Import dependencies

In [2]:
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
%matplotlib widget

In [4]:
import os
import sys
sys.path.insert(1, os.getcwd()+'/src')
sys.path.insert(1, os.getcwd()+'/dsge')
import time
from datetime import datetime
import numpy as np
from gp_model import GPModel
from ppbo_settings import PPBO_settings
from acquisition import next_query
from gui import GUI_session
from jupyter_ui_poll import run_ui_poll_loop
from ipywidgets.widgets import HBox
from IPython.display import display, clear_output
from IPython.core.display import display, HTML
display(HTML("<style>div.output_scroll { height: 60em; }</style>")) #Make outputwindow larger
from priorelicitation import Prior, g_theta, minimize_KL, sample_h, plot
from random_fourier_sampler import Hsampler
from scipy.stats import norm, beta
from matplotlib import pyplot as plt

#### Specify the aquisition strategy and the problem setting
Acquisition startegies with unit projections ($\boldsymbol{\xi}$ is an unit vector):
- PCD = preferential coordinate descent
- EI-EXT = same as EI-FIXEDX except only unit projections are allowed
- EI-EXT-FAST = same as EI-EXT except $d\mathbf{x}$ integral omitted
- EI-VARMAX = same as EI-EXT except $\mathbf{x}$ is chosen to maximize GP variance
- EI-VARMAX-FAST = same as EI-VARMAX except $d\mathbf{x}$ integral omitted

Acquisition startegies with non-unit projections:
- EI = expected improvement by projective preferential query
- EI-FIXEDX = same as EI except $\mathbf{x}$ is fixed to $\textrm{argmax}_{\mathbf{x}}\mu(\mathbf{x})$ (xstar)
- EXT = pure exploitation
- EXR = pure exploration (variance maximization)
- RAND = random 

'user_feedback_grid_size' determines the granularity of the slider, that is, the number of options available for the user. Each option is run in a dedicated CPU. Hence, the optimal performance will be achieved if 'user_feedback_grid_size' is a multiple of the number of CPUs. 

In [5]:
acquisition_strategy = 'PCD'

In [6]:
PPBOset = PPBO_settings(D=7,bounds=((0.3,0.8),(0.4,0.9),(0.3,0.8),(0.3,0.8),(2,8),(0.5,3),(0.3,0.8)),
                    xi_acquisition_function=acquisition_strategy,user_feedback_grid_size=7,#15
                    theta_initial=[0.05,0.27,0.2],m=25,verbose=False,
                    skip_computations_during_initialization=True,alpha_grid_distribution='TGN')

#### Set initial queries

In [7]:
initial_queries_xi = np.diag([PPBOset.original_bounds[i][1] for i in range(PPBOset.D)])
initial_queries_x = np.random.uniform([PPBOset.original_bounds[i][0] for i in range(PPBOset.D)], [PPBOset.original_bounds[i][1] for i in range(PPBOset.D)], (len(initial_queries_xi), PPBOset.D))

#### Querying settings

In [8]:
NUMBER_OF_QUERIES = 14
ADAPTIVE_INITIALIZATION = True  #In initilization: immediatly update the coordinate according to the user feedback

#### Hyperparameter optimization

In [9]:
OPTIMIZE_HYPERPARAMETERS_AFTER_EACH_ITERATION = False
OPTIMIZE_HYPERPARAMETERS_AFTER_QUERY_NUMBER = 1000 

#### Initialize the user session

In [10]:
should_log = False
if should_log:
    orig_stdout = sys.stdout
    log_file = open('dsge/user_session_log_'+str(SESSION_NAME)+'_'+str(datetime.now().strftime("%d-%m-%Y_%H-%M-%S"))+'.txt', "w")
    sys.stdout = log_file
GUI_ses = GUI_session(PPBOset)
results_mustar = []
results_xstar = []
results_xstar_unscaled = []

## Elicitation loop

In [11]:
start = time.time() 
for i in range(len(initial_queries_xi)+NUMBER_OF_QUERIES):
    if i < len(initial_queries_xi):
        print(f'Initialization in progress... ({i+1}/{len(initial_queries_xi)})\r', end="")
        if i==len(initial_queries_xi)-1:
            GP_model.turn_initialization_off()
    else:
        print(f'Elicitation in progress... ({i+1-len(initial_queries_xi)}/{NUMBER_OF_QUERIES})\r', end="")
        if i+1==len(initial_queries_xi)+NUMBER_OF_QUERIES:
            GP_model.set_last_iteration()
    ''' Next projective preferential query '''
    if i < len(initial_queries_xi):
        xi = initial_queries_xi[i].copy()
        if not i==0 and GUI_ses.user_feedback is not None and ADAPTIVE_INITIALIZATION:
            initial_queries_x[i:,:] = GUI_ses.user_feedback
        x = initial_queries_x[i].copy()
        x[xi!=0] = 0
    else:
        xi,x = next_query(PPBOset,GP_model,unscale=True)
    GUI_ses.set_x(x)
    GUI_ses.set_xi(xi)
    ''' Event loop '''
    button,slider,fig,l1,l2,l3,l4 = GUI_ses.prepare_app()
    app = HBox(children=(slider,button))
    def wait_user_input():
        if not GUI_ses.user_feedback_was_given:
            GUI_ses.update_plot(l1,l2,l3,l4,fig,slider)  
            return None
        app.close()
        GUI_ses.user_feedback_was_given = False
        GUI_ses.save_results()
        return 1       
    display(app)
    dt = run_ui_poll_loop(wait_user_input)
    ''' Create GP model for first time '''
    if i==0:
        GP_model = GPModel(PPBOset)
        GP_model.COVARIANCE_SHRINKAGE = 1e-6
    ''' Update GP model '''
    GP_model.update_feedback_processing_object(np.array(GUI_ses.results))
    GP_model.update_data()
    GP_model.update_model()
    if i+1==OPTIMIZE_HYPERPARAMETERS_AFTER_QUERY_NUMBER:
        GP_model.update_model(optimize_theta=True)     
    else:
        GP_model.update_model(optimize_theta=OPTIMIZE_HYPERPARAMETERS_AFTER_EACH_ITERATION)
    ''' Save the predictive mean maximum and maximizer'''
    results_mustar.append(GP_model.mustar)
    results_xstar.append(GP_model.xstar)
    results_xstar_unscaled.append(GP_model.FP.unscale(GP_model.xstar))
    clear_output(wait=True)
print("Elicitation done!")

Elicitation done!


In [12]:
print("Total time: " + str(time.time()-start) + " seconds.")

Total time: 71.56457042694092 seconds.


## Save the results

#### Save the user session results

In [13]:
#Save results to csv-file
print("Saving the user session results...")
user_session_results = GUI_ses.results.copy()
user_session_results['iter_mustar'] = results_mustar
user_session_results['iter_xstar'] = results_xstar
user_session_results['iter_xstar_unscaled'] = results_xstar_unscaled
user_session_results.to_csv('dsge/user_session_results_'+str(SESSION_NAME)+'_'+str(datetime.now().strftime("%d-%m-%Y_%H-%M-%S"))+'.csv',index=False)
#Close the log-file
if should_log:
    sys.stdout = orig_stdout
    log_file.close()

Saving the user session results...


## Analyze the results

#### The user's most preferred hyperprior configuration

In [14]:
print('Preferred lambda: ' + str(GP_model.FP.unscale(GP_model.xstar)))

Preferred lambda: [0.3        0.44707035 0.50024382 0.49999491 5.00013971 1.00012317
 0.50022839]


#### Find parametric priors that are in KL-sense closest to the expert's opinion

In [15]:
h_sampler = Hsampler(GP_model,nFeatures=1500)
h_sampler.generate_basis()
h_sampler.update_phi_X()
h_sampler.update_omega_MAP()
h_sampler.update_covariancematrix()

In [16]:
lambda_sample=[]
for i in range(200):
    lambda_sample.append(GP_model.FP.unscale(h_sampler.sample_xstar()))
lambda_sample = np.array(lambda_sample)

In [17]:
print(np.mean(lambda_sample,axis=0))

[0.38129176 0.5265668  0.50995322 0.52148954 5.07371802 1.27165425
 0.52608163]


In [18]:
print(np.std(lambda_sample,axis=0))

[0.12016709 0.13435517 0.14427049 0.13540297 1.68588009 0.70898942
 0.13624773]


In [19]:
fixed_hyperparams = [0.1,0.1,0.2,0.2,1.5,0.375,0.1] #Scale parameters of the priors
prior0 = Prior(family='beta',valrange=(0,1),lambda_indices=[0],fixed_hyperparams=[fixed_hyperparams[0]],is_location_param=True)
prior1 = Prior(family='beta',valrange=(0,1),lambda_indices=[1],fixed_hyperparams=[fixed_hyperparams[1]],is_location_param=True) 
prior2 = Prior(family='beta',valrange=(0,1),lambda_indices=[2],fixed_hyperparams=[fixed_hyperparams[2]],is_location_param=True) 
prior3 = Prior(family='beta',valrange=(0,1),lambda_indices=[3],fixed_hyperparams=[fixed_hyperparams[3]],is_location_param=True) 
prior4 = Prior(family='normal',valrange=(2,15),lambda_indices=[4],fixed_hyperparams=[fixed_hyperparams[4]],is_location_param=True) 
prior5 = Prior(family='normal',valrange=(0.25,3),lambda_indices=[5],fixed_hyperparams=[fixed_hyperparams[5]],is_location_param=True) 
prior6 = Prior(family='beta',valrange=(0,1),lambda_indices=[6],fixed_hyperparams=[fixed_hyperparams[6]],is_location_param=True)
optimal_hyperparameters = []

Prior0: Beta(x,0.1) for parameter $\zeta_{w}$

In [20]:
thetas = np.arange(prior0.domain[0], prior0.domain[1], 0.01)
g=[]
for theta in thetas:
    g.append(g_theta(theta,prior0,lambda_sample))
q = beta.pdf
opt_hyperparam = minimize_KL(q,g,prior0)
optimal_hyperparameters.append(opt_hyperparam)

In [21]:
#Plot parametric and non-parametric priors elicited from the user
plot(q(thetas, opt_hyperparam, prior0.fixed_hyperparams[0]),g,thetas,fig_id=100)
#Plot true target (posterior estimated on the extended sample 1984-2015)
plt.plot(thetas, q(thetas, 0.788, prior0.fixed_hyperparams[0]), c='green',linestyle='dashed')
plt.ylim(0, 2),plt.legend(['elicited parametric','elicited non-parametric','target shown to expert']);
plt.savefig('dsge/prior0_'+str(SESSION_NAME)+'.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Prior1: Beta(x,0.1) for parameter $\zeta_{p}$

In [22]:
thetas = np.arange(prior1.domain[0], prior1.domain[1], 0.01)
g=[]
for theta in thetas:
    g.append(g_theta(theta,prior1,lambda_sample))
q = beta.pdf
opt_hyperparam = minimize_KL(q,g,prior1)
optimal_hyperparameters.append(opt_hyperparam)

In [23]:
#Plot parametric and non-parametric priors elicited from the user
plot(q(thetas, opt_hyperparam, prior1.fixed_hyperparams[0]),g,thetas,fig_id=11)
#Plot true target (posterior estimated on the extended sample 1984-2015)
plt.plot(thetas, q(thetas, 0.861, prior1.fixed_hyperparams[0]), c='green',linestyle='dashed')
plt.ylim(0, 2),plt.legend(['elicited parametric','elicited non-parametric','target shown to expert']);
plt.savefig('dsge/prior1_'+str(SESSION_NAME)+'.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Prior2: Beta(x,0.2) for parameter $\theta_{p}$

In [24]:
thetas = np.arange(prior2.domain[0], prior2.domain[1], 0.01)
g=[]
for theta in thetas:
    g.append(g_theta(theta,prior2,lambda_sample))
q = beta.pdf
opt_hyperparam = minimize_KL(q,g,prior2)
optimal_hyperparameters.append(opt_hyperparam)

In [25]:
#Plot parametric and non-parametric priors elicited from the user
plot(q(thetas, opt_hyperparam, prior2.fixed_hyperparams[0]),g,thetas,fig_id=12)
#Plot true target (posterior estimated on the extended sample 1984-2015)
plt.plot(thetas, q(thetas, 0.648, prior2.fixed_hyperparams[0]), c='green',linestyle='dashed')
plt.ylim(0, 2),plt.legend(['elicited parametric','elicited non-parametric','target shown to expert']);
plt.savefig('dsge/prior2_'+str(SESSION_NAME)+'.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Prior3: Beta(x,0.2) for parameter $\theta_{w}$

In [26]:
thetas = np.arange(prior3.domain[0], prior3.domain[1], 0.01)
g=[]
for theta in thetas:
    g.append(g_theta(theta,prior3,lambda_sample))
q = beta.pdf
opt_hyperparam = minimize_KL(q,g,prior3)
optimal_hyperparameters.append(opt_hyperparam)

In [27]:
#Plot parametric and non-parametric priors elicited from the user
plot(q(thetas, opt_hyperparam, prior3.fixed_hyperparams[0]),g,thetas,fig_id=13)
#Plot true target (posterior estimated on the extended sample 1984-2015)
plt.plot(thetas, q(thetas, 0.649, prior3.fixed_hyperparams[0]), c='green',linestyle='dashed')
plt.ylim(0, 2),plt.legend(['elicited parametric','elicited non-parametric','target shown to expert']);
plt.savefig('dsge/prior3_'+str(SESSION_NAME)+'.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Prior4: N(x,1.5) for parameter inv_adj_cost

In [28]:
thetas = np.arange(prior4.domain[0], prior4.domain[1], 0.01)
g=[]
for theta in thetas:
    g.append(g_theta(theta,prior4,lambda_sample))
q = norm.pdf
opt_hyperparam = minimize_KL(q,g,prior4)
optimal_hyperparameters.append(opt_hyperparam)

In [29]:
#Plot parametric and non-parametric priors elicited from the user
plot(q(thetas, opt_hyperparam, prior4.fixed_hyperparams[0]),g,thetas,fig_id=14)
#Plot true target (posterior estimated on the extended sample 1984-2015)
plt.plot(thetas, q(thetas, 5.632, prior4.fixed_hyperparams[0]), c='green',linestyle='dashed')
plt.ylim(0, 0.4),plt.legend(['elicited parametric','elicited non-parametric','target shown to expert']);
plt.savefig('dsge/prior4_'+str(SESSION_NAME)+'.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Prior5: N(x,0.375) for parameter $\sigma_c$

In [30]:
thetas = np.arange(prior5.domain[0], prior5.domain[1], 0.01)
g=[]
for theta in thetas:
    g.append(g_theta(theta,prior5,lambda_sample))
q = norm.pdf
opt_hyperparam = minimize_KL(q,g,prior5)
optimal_hyperparameters.append(opt_hyperparam)

In [31]:
#Plot parametric and non-parametric priors elicited from the user
plot(q(thetas, opt_hyperparam, prior5.fixed_hyperparams[0]),g,thetas,fig_id=15)
#Plot true target (posterior estimated on the extended sample 1984-2015)
plt.plot(thetas, q(thetas, 1.267, prior5.fixed_hyperparams[0]), c='green',linestyle='dashed')
plt.ylim(0, 1.5),plt.legend(['elicited parametric','elicited non-parametric','target shown to expert']);
plt.savefig('dsge/prior5_'+str(SESSION_NAME)+'.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Prior6: Beta(x,0.1) for parameter $h$

In [32]:
thetas = np.arange(prior6.domain[0], prior6.domain[1], 0.01)
g=[]
for theta in thetas:
    g.append(g_theta(theta,prior6,lambda_sample))
q = beta.pdf
opt_hyperparam = minimize_KL(q,g,prior6)
optimal_hyperparameters.append(opt_hyperparam)

In [33]:
#Plot parametric and non-parametric priors elicited from the user
plot(q(thetas, opt_hyperparam, prior6.fixed_hyperparams[0]),g,thetas,fig_id=16)
#Plot true target (posterior estimated on the extended sample 1984-2015)
plt.plot(thetas, q(thetas, 0.504, prior6.fixed_hyperparams[0]), c='green',linestyle='dashed')
plt.ylim(0, 1),plt.legend(['elicited parametric','elicited non-parametric','target shown to expert']);
plt.savefig('dsge/prior6_'+str(SESSION_NAME)+'.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Save the hyperparameters obtained from the KL-minimization

In [34]:
with open('dsge/optimal_hyperparameters_'+str(SESSION_NAME)+'.txt', 'w') as f:
    f.write(str(optimal_hyperparameters))

In [35]:
print('Thank you for your effort! You can logout now.')

Thank you for your effort! You can logout now.
